In [1]:
import sys
sys.path.append('..')
from osp import *

In [2]:
df_jstor = get_jstor_data().fillna('')

In [3]:
df_pmla = get_pmla_df().fillna('')

100%|██████████| 71902/71902 [00:02<00:00, 29008.23it/s]


In [4]:
df_pmla.columns

Index(['creator', 'datePublished', 'docSubType', 'docType', 'fullText', 'id',
       'identifier', 'isPartOf', 'issueNumber', 'language', 'outputFormat',
       'pageCount', 'pageEnd', 'pageStart', 'pagination', 'provider',
       'publicationYear', 'publisher', 'sourceCategory', 'title', 'url',
       'volumeNumber', 'wordCount', 'tdmCategory', 'abstract', 'subTitle',
       'year', 'decade'],
      dtype='object')

In [5]:
df_pmla.iloc[0]

creator                                      [Jean-Jacques Demorest]
datePublished                                             1967-05-01
docSubType                                          research-article
docType                                                      article
fullText           [PASCAL ET LE DESEQUILIBRE PAR JEAN-JACQUES DE...
id                                http://www.jstor.org/stable/461288
identifier         [{'name': 'issn', 'value': '00308129'}, {'name...
isPartOf                                                        PMLA
issueNumber                                                        2
language                                                       [fre]
outputFormat                              [unigram, bigram, trigram]
pageCount                                                          6
pageEnd                                                          196
pageStart                                                        191
pagination                        

In [54]:
def format_url(url):
    x = url.split(".")
    return ".".join(x[1:])


def pmla2row(row):
    ids = row['identifier']
    uuid = [d['value'] for d in ids if d['name'] == 'local_uuid']
    uuid = uuid[0] if uuid else None
    id = "lit/" + row['url'].split('/stable/')[-1]
    return {
        'id': id,
        "uuid": uuid,
        'title': row['title'],
        'author': ';'.join(row['creator']),
        'year': int(row['datePublished'].split('-')[0]),
        'journal': row['isPartOf'],
        'volume': row['volumeNumber'],
        'issue': row['issueNumber'],
        'url': format_url(row['url']),
        'publisher': row['publisher'],
        'discipline': "Literature"
    }

In [55]:
pmla2row(df_pmla.iloc[0])

{'id': 'lit/461288',
 'uuid': 'bf1701af-c700-36b5-ad4e-8d30c252f7a1',
 'title': 'Pascal et le déséquilibre',
 'author': 'Jean-Jacques Demorest',
 'year': 1967,
 'journal': 'PMLA',
 'volume': '82',
 'issue': '2',
 'url': 'jstor.org/stable/461288',
 'publisher': 'Modern Language Association',
 'discipline': 'Literature'}

In [56]:
df_jstor.iloc[0]

id                                    f6eecd30-8c4a-3c3d-a9da-4e777d091b2c
review_required                                                       True
ithaka_doi                                                10.2307/40231690
identifiers              {'print_isbn': None, 'online_isbn': None, 'pri...
title                                            "Aristotlés" Horror Vacui
is_part_of                                  Canadian Journal of Philosophy
creators_string                                                 John Thorp
creators                 [{'first_name': 'John', 'last_name': 'Thorp', ...
publishers                                    [Cambridge University Press]
published_date                                                   1990-06-0
languages                                                            [eng]
discipline_names                                              [Philosophy]
issue_number                                                             2
issue_volume             

In [57]:

def jstor2row(row):
    uuid = row['id']
    id = "phil/" + row['url'].split('/stable/')[-1]
    return {
        'id': id,
        "uuid": uuid,
        'title': row['title'],
        'author': row['creators_string'],
        'year': int(row['published_date'].split('-')[0]),
        'journal': row['is_part_of'],
        'volume': row['issue_volume'],
        'issue': row['issue_number'],
        'url': format_url(row['url']),
        'publisher': '; '.join(row['publishers']),
        'discipline': "Philosophy"
    }

In [58]:
jstor2row(df_jstor.iloc[0])

{'id': 'phil/10.2307/40231690',
 'uuid': 'f6eecd30-8c4a-3c3d-a9da-4e777d091b2c',
 'title': '"Aristotlés" Horror Vacui',
 'author': 'John Thorp',
 'year': 1990,
 'journal': 'Canadian Journal of Philosophy',
 'volume': '20',
 'issue': '2',
 'url': 'jstor.org/stable/10.2307/40231690',
 'publisher': 'Cambridge University Press',
 'discipline': 'Philosophy'}

In [59]:
ld = []
for i,row in df_jstor.iterrows():
    ld.append(jstor2row(row))
for i,row in df_pmla.iterrows():
    ld.append(pmla2row(row))
df = pd.DataFrame(ld)
df

,id,uuid,title,author,year,journal,volume,issue,url,publisher,discipline
0,phil/10.2307/40231690,f6eecd30-8c4a-3c3d-a9da-4e777d091b2c,"""Aristotlés"" Horror Vacui",John Thorp,1990,Canadian Journal of Philosophy,20,2,jstor.org/stable/10.2307/40231690,Cambridge University Press,Philosophy
1,phil/10.2307/40230399,aaff574b-da8a-389b-b82c-8bcfbf981a8e,"""None in Particular""",John Woods,1973,Canadian Journal of Philosophy,2,3,jstor.org/stable/10.2307/40230399,Cambridge University Press,Philosophy
2,phil/10.2307/40231533,9a99d15f-bbe2-349e-b5af-cf96fd35c6eb,"""Tractatus"" 2.022-2.023",Raymond D. Bradley,1987,Canadian Journal of Philosophy,17,2,jstor.org/stable/10.2307/40231533,Cambridge University Press,Philosophy
3,phil/10.2307/40230622,9af0da08-8130-3178-86cb-5df65d6fb0cd,"""Tractatus"" 5.54-5.5422",Eric B. Dayton,1976,Canadian Journal of Philosophy,6,2,jstor.org/stable/10.2307/40230622,Cambridge University Press,Philosophy
4,phil/10.2307/40231225,041ecb2f-f8ab-398e-a1df-8813eaffa841,"'Can,' Compatibilism, and Possible Worlds",Michael J. Zimmerman,1981,Canadian Journal of Philosophy,11,4,jstor.org/stable/10.2307/40231225,Cambridge University Press,Philosophy
...,...,...,...,...,...,...,...,...,...,...,...
58121,lit/2873182,None,The Subversive Discourse of the Wife of Bath: ...,Barrie Ruth Straus,1988,ELH,55,3,jstor.org/stable/2873182,Johns Hopkins University Press,Literature
58122,lit/459445,9a9690d3-bcdb-3274-8ec1-8e50da89d1bf,"Shelley's ""Hymn to Intellectual Beauty""",Elizabeth Nitchie,1948,PMLA,63,2,jstor.org/stable/459445,Modern Language Association,Literature
58123,lit/3715352,None,"The Water-Bridge in Chrétien's ""Charrette""",K. G. T. Webster,1931,The Modern Language Review,26,1,jstor.org/stable/3715352,Modern Humanities Research Association,Literature
58124,lit/2871890,None,The Creation of the Self in Gerard Manley Hopkins,J. Hillis Miller,1955,ELH,22,4,jstor.org/stable/2871890,Johns Hopkins University Press,Literature


In [60]:
df.sample(10)

,id,uuid,title,author,year,journal,volume,issue,url,publisher,discipline
31384,phil/10.2307/20124691,a42cef37-f91f-3259-91b5-9332652fd672,Greek Historians,Leo Strauss,1968,The Review of Metaphysics,21,4,jstor.org/stable/10.2307/20124691,Philosophy Education Society Inc.,Philosophy
46016,lit/468516,22dd9b6d-ef8a-3892-8b6c-335348471d15,Literary Discourse and the Social Historian,Brian Stock,1977,New Literary History,8,2,jstor.org/stable/468516,Johns Hopkins University Press,Literature
11645,phil/10.2307/2108473,38f67446-daa7-350d-899c-8c6e1a1ef0f9,Radical Anti-Deflationism,Peter S. Dillard,1996,Philosophy and Phenomenological Research,56,1,jstor.org/stable/10.2307/2108473,Wiley; International Phenomenological Society;...,Philosophy
33602,lit/30030149,None,On Truth and Lie in a Colonial Sense: Kipling'...,Ambreen Hai,1997,ELH,64,2,jstor.org/stable/30030149,Johns Hopkins University Press,Literature
17512,phil/10.2307/41932114,11164863-7018-353a-966f-1b7e90be70ad,Embedding philosophers in the practices of sci...,Nancy Tuana,2013,Synthese,190,11,jstor.org/stable/10.2307/41932114,Springer,Philosophy
8992,phil/10.2307/2377492,f4228493-6db4-3373-8aaa-efddc025e9b6,The Modern World-Order and the Original Nature...,Daniel Bell Leary,1922,International Journal of Ethics,32,3,jstor.org/stable/10.2307/2377492,The University of Chicago Press,Philosophy
36902,lit/458408,094545f1-083b-33d1-82d2-e90da65d5fa7,A Woman Killed with Kindness,Hallett D. Smith,1938,PMLA,53,1,jstor.org/stable/458408,Modern Language Association,Literature
32363,phil/10.2307/20123859,2773b54b-acdd-3f4c-ac5b-11a3f0de363b,The Elements of the Physical Universe,Paul Weiss,1961,The Review of Metaphysics,15,1,jstor.org/stable/10.2307/20123859,Philosophy Education Society Inc.,Philosophy
36388,lit/20057975,aa0d5a8f-13d8-3a52-8cd2-126a943893ab,Moving Tales: Narrative Drift in Oral Culture ...,Benjamin Griffin,2006,New Literary History,37,4,jstor.org/stable/20057975,Johns Hopkins University Press,Literature
35309,lit/518845,None,"Clarissa's Death, Clarissa's Sale, and the Tex...",Tom Keymer,1994,The Review of English Studies,45,179,jstor.org/stable/518845,Oxford University Press,Literature


In [66]:
def save_jstor_txt(df):
    for i,row in tqdm(df.iterrows(), total=len(df)):
        d = jstor2row(row)
        fn = '../data/txt/' + d['id'] + '.txt'
        fldr = os.path.dirname(fn)
        os.makedirs(fldr, exist_ok=True)
        
        txt = '\n\n\n'.join(row['full_text'])
        with open(fn, 'w') as f:
            f.write(txt)


In [67]:
save_jstor_txt(df_jstor)

100%|██████████| 32783/32783 [00:04<00:00, 6779.11it/s]


In [71]:
def save_pmla_txt(df):
    for i,row in tqdm(df.iterrows(), total=len(df)):
        d = pmla2row(row)
        fn = '../data/txt/' + d['id'] + '.txt'
        fldr = os.path.dirname(fn)
        os.makedirs(fldr, exist_ok=True)
        
        txt = '\n\n\n'.join(row['fullText'])
        with open(fn, 'w') as f:
            f.write(txt)
    return

In [72]:
save_pmla_txt(df_pmla)

100%|██████████| 25343/25343 [00:03<00:00, 7232.08it/s]


In [74]:
df.set_index('id').to_csv('../data/metadata.csv')